# Transformation

In [5]:
# Imports
import pandas as pd

In [12]:
# Paths
films_path = "../extraction/extracted_data/films_data.csv"
oscars_path = "../extraction/extracted_data/oscars_data.csv"
streaming_path = "../extraction/extracted_data/streaming_data.csv"

films_df = pd.read_csv(films_path)
oscars_df = pd.read_csv(oscars_path)
streaming_df = pd.read_csv(streaming_path)